# Instanzen-Suche
Mit diesem Notebook können alle Instanzen aufgelistet werden, die bestimmte Bedingungen erfüllen. Die Bedingungen können frei gewählt werden. Sie bestehen aus einer Annotationskategorie und einer weiteren Bedingung. Folgende Suchparameter gibt es:
+ nach Wörtern (Lexemen)
+ Nach Wortarten oder Wortartenlisten (benutzt wird das [Stuttgart-Tübingen-Tagset](https://homepage.ruhr-uni-bochum.de/stephen.berman/Korpuslinguistik/Tagsets-STTS.html) oder [Universal POS tags](https://universaldependencies.org/u/pos/))
+ Nach Anzahl vergebener Kategorietags (z.B.: 3 Moralwerte)
+ Nach bestimmten Annotationslabels (z.B.: Forderer)

Bezüglich der Annotationskategorien stehen Folgende zur Verfügung:
+ *obj_morals*: Alle Moralwerte, die keine subjektiven Ausdrücke sind
+ *subj_morals*: Moralwerte, die subjektive Ausdrücke sind
+ *all_morals*: Alle Moralwerte
+ *protagonists*: Alle Protagonisten-Token
+ *protagonists_doubles*: Alle Protagonisten-Rollen (Achtung: manche Protagonisten-Token werden dabei doppelt gezählt - nämlich, wenn sie mehrere Rollen haben)
+ *com_functions*: Kommunikative Funktionen
+ *expl_demands*: Explizite Forderungen
+ *impl_demands*: Implizite Forderungen
+ *all_demands*: Alle Forderungen

## Setup

Muss ausgeführt werden, wenn das Notebook in Colab ausgeführt wird.
Das Laden der Spacy-Modelle kann länger dauern; man kann es beschleunigen, indem man nur die Sprachen läd, die man benötigt (und die anderen Befehlszeilen löscht).

In [1]:
import os

if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    %git clone https://github.com/maria-becker/Moralization/
    %pip install HanTa
    %pip install xlsxwriter
    %pip install spacy

    %python -m spacy download de_core_web_lg
    %python -m spacy download en_core_web_lg
    %python -m spacy download fr_core_web_lg
    %python -m spacy download it_core_web_lg

    %cd "/content/Moralization/Annotation Analysis Tools/data_analysis"
else:
    print("This code should be run in Google Colab only.")

This code should be run in Google Colab only.


Die import-Statements müssen immer ausgeführt werden.

In [2]:
from analysis_functions import (
    filtering_corpus,
    _util_ as util,
    _corpus_extraction_ as ce
)

## Suchfunktionen (Einzelte Korpusdateien)
Code für die Analyse einzelner XMI-Dateien.

Hier bitte eine Liste *file_list* erstellen, die die Dateipfade zu allen Korpusdateien enthält, die ihr analysieren wollt.
+ *Am einfachsten erhält man einen Dateipfad, indem man die Datei im Ordersystem rechtsklickt und dann -> 'Pfad kopieren' auswählt. Und unbedingt den Pfad in Anführungszeichen setzen und davor ein kleines "r"!*
+ *Wer sich ein bisschen auskennt, kann zum Erstellen der Liste auch das os-Modul benutzen, das aus diesem Grund oben importiert wurde.*

Bei der Erstellung des corpus-Objekts ist zu beachten, welche Sprache im Korpus vertreten ist.
    + Deutsch ('de')
    + Englisch ('en')
    + Französisch ('fr')
    + Italienisch ('it')
Mehrsprachliche Korpora werden nicht unterstützt.

In [3]:
filepaths = [
    r"/home/brunobrocai/Desktop/Gerichtsurteile-pos-AW-neu-optimiert-BB_vergoldet.xmi",
]
corpus_collection = ce.Corpus(filepaths, 'ger')

Mit der folgenden Funktion kann nach Lexemen, die als eine bestimmte Kategorie annotiert wurden, gesucht werden.

Parameter:
+ Der erste Parameter ist die Korpussammlumg. Hier einfach *corpus_collection* stehen lassen.
+ Der zweite Parameter ist das Lexem, das gesucht werden soll
+ Der dritte Parameter ist die Kategorie, in der die Wortart gesucht werden soll, s.o.
+ Der vorletzte Parameter bestimmt, ob HanTa (*True*) oder Spacy zum Lemmatisieren eingesetzt werden soll. **Achtung: HanTa eignet sich nur für Deutsch und Englisch (erzielt dort aber evtl. bessere Ergebnisse)!**
+ Der letzte Parameter *export* (*True* oder *False*) gibt an, ob die Daten als Excel-Datei im derzeitigen Verzeichnis abgelegt werden sollen.

In [4]:
word_labels = filtering_corpus.lemmata_category_instances(
    corpus_collection,
    "Menschenwürde",
    "all_morals",
    tagger="Spacy",
    export=False
)
util.print_subcorp_dict(word_labels)

GERICHTSURTEILE-POS-AW-NEU-OPTIMIERT-BB_VERGOLDET.XMI
„Sie war und ist eine Organisation, die Bestrebungen verfolgt, die mit unserer Verfassungsordnung nicht vereinbar sind”, sagte Rech. Milli Görüs sei gegen die freiheitliche demokratische Grundordnung gerichtet und stehe in krassem Gegensatz zu den Grundrechten, DER MENSCHENWÜRDE, der Gleichberechtigung von Mann und Frau sowie der Religionsfreiheit.
„DIE MENSCHENWÜRDE ist migrationspolitisch nicht zu relativieren“, heißt es in der Entscheidung.
Allerdings sieht das Gericht in den bisherigen Bestimmungen auch EINE VERLETZUNG DER MENSCHENWÜRDE UND DES SOZIALSTAATSPRINZIPS. Vor allem wohl deshalb, weil versäumt wurde, den genauen Bedarf von Kindern zu ermitteln und auch zu definieren.
Der Mann fand dies unzumutbar, weil ihm weniger als das Existenzminimum verbleibe. Seine Rechte auf MENSCHENWÜRDE und körperliche Unversehrtheit seien damit verletzt. Diese Sichtweise lehnten die Richter ab.
Ein solches Arbeitsentgelt verstoße gegen DIE GR

Mit der folgenden Funktion kann nach Wortarten, die als eine bestimmte Kategorie annotiert wurden, gesucht werden.
Wird mit HanTa (s.u.) getaggt, nutzt die Funktion das [Stuttgart-Tübingen-Tagset](https://homepage.ruhr-uni-bochum.de/stephen.berman/Korpuslinguistik/Tagsets-STTS.html). Andernfalls - also mit Spacy - wird [Universal POS tags](https://universaldependencies.org/u/pos/) benutzt, das etwas allgemeiner ist.

Parameter:
+ Der erste Parameter ist die Korpussammlumg. Hier einfach *corpus_collection* stehen lassen.
+ Der zweite Parameter ist die Wortart, die gesucht werden soll
+ Der dritte Parameter ist die Kategorie, in der das Lexem gesucht werden soll, s.o.
+ Der vorletzte Parameter bestimmt, ob HanTa (*True*) oder Spacy zum Lemmatisieren eingesetzt werden soll. **Achtung: HanTa eignet sich nur für Deutsch und Englisch (erzielt dort aber evtl. bessere Ergebnisse)!**
+ Der letzte Parameter *export* (*True* oder *False*) gibt an, ob die Daten als Excel-Datei im derzeitigen Verzeichnis abgelegt werden sollen.

In [5]:
pos_labels = filtering_corpus.pos_category_instances(
    corpus_collection,
    "PROPN",
    "protagonists",
    tagger="Spacy",
    export=False
)
util.print_subcorp_dict(pos_labels)

KeyError: {'Coordinates': (183764, 183816), 'Rolle': 'Forderer:in', 'Gruppe': 'Individuum', 'own/other': None} -- Is the label inside a moralization?
GERICHTSURTEILE-POS-AW-NEU-OPTIMIERT-BB_VERGOLDET.XMI
DIE FAZ argumentiert, die Methode schaffe nicht genug Transparenz für die Werbewirtschaft, DIE IVW wende eigene Regeln zur gemeinsamen Auflagenausweisung nicht korrekt an. TOBIAS TREVISAN, SPRECHER DER FAZ-GESCHÄFTSFÜHRUNG, sagt: „Wir bieten der Werbewirtschaft eine Währung an und diese Währung verliert durch diese Ausweisungspraxis an Glaubwürdigkeit“.
Zum Schutz der Bevölkerung" sei dies dringend notwendig, so BECKSTEIN.
GERICHTSPRÄSIDENT MICHAEL BERTRAMS stellte in der Verhandlung bereits fest, dass das Sondervermögen für die WestLB „faktisch nicht gebraucht“ werde. Er erinnerte außerdem an das Wirtschaftlichkeitsgebot der Verfassung, es „gründe in der Generationengerechtigkeit, dass auch künftige Generationen über Haushaltsspielraum verfügen“.
Auch DER VIZE-FRAKTIONSVORSITZENDE DER

Mit der folgenden Funktion kann nach einer Liste von Wortarten, die als eine bestimmte Kategorie annotiert wurden, gesucht werden. Dies ist vor allem daher sinnvoll, weil bestimmten Wortarten in den Tagsets mehr als ein Tag zugeordnet ist.
Wird mit HanTa (s.u.) getaggt, nutzt die Funktion das [Stuttgart-Tübingen-Tagset](https://homepage.ruhr-uni-bochum.de/stephen.berman/Korpuslinguistik/Tagsets-STTS.html). Andernfalls - also mit Spacy - wird [Universal POS tags](https://universaldependencies.org/u/pos/) benutzt, das etwas allgemeiner ist.

Parameter:
+ Der erste Parameter ist die Korpussammlumg. Hier einfach *corpus_collection* stehen lassen.
+ Der zweite Parameter ist die Wortart, die gesucht werden soll
+ Der dritte Parameter ist die Kategorie, in der die Wortart gesucht werden soll, s.o.
+ Der vorletzte Parameter bestimmt, ob HanTa (*True*) oder Spacy zum Lemmatisieren eingesetzt werden soll. **Achtung: HanTa eignet sich nur für Deutsch und Englisch (erzielt dort aber evtl. bessere Ergebnisse)!**
+ Der letzte Parameter *export* (*True* oder *False*) gibt an, ob die Daten als Excel-Datei im derzeitigen Verzeichnis abgelegt werden sollen.

In [6]:
poslist_labels = filtering_corpus.poslist_category_instances(
    corpus_collection,
    ["NE", "NN"],
    "protagonists",
    tagger="HanTa",
    export=False
)
util.print_subcorp_dict(poslist_labels)

KeyError: {'Coordinates': (183764, 183816), 'Rolle': 'Forderer:in', 'Gruppe': 'Individuum', 'own/other': None} -- Is the label inside a moralization?
GERICHTSURTEILE-POS-AW-NEU-OPTIMIERT-BB_VERGOLDET.XMI
DIE FAZ argumentiert, die Methode schaffe nicht genug Transparenz für die Werbewirtschaft, DIE IVW wende eigene Regeln zur gemeinsamen Auflagenausweisung nicht korrekt an. TOBIAS TREVISAN, SPRECHER DER FAZ-GESCHÄFTSFÜHRUNG, sagt: „Wir bieten der Werbewirtschaft eine Währung an und diese Währung verliert durch diese Ausweisungspraxis an Glaubwürdigkeit“.
DFL-GESCHÄFTSFÜHRER HOLGER HIERONYMUS ergänzte: "Die Sicherheit DES ZUSCHAUERS ist das höchste Gut."
DER EUGH ist hingegen der Ansicht, dass sich DIE ÖFFENTLICHKEIT über mögliche Gefahren für die Umwelt und die menschliche Gesundheit informieren können
"Es ist das oberste Gebot eines jeden STAATES, SEINE KINDER zu schützen", kommentierte DER RICHTER gestern sein Urteil.
DAS OLG bestätigte die Anusinspektion als rechtmäßig und zumutbar, 

Mit der folgenden Funktion kann nach Anzahlen von Kategorie-Vergaben gesucht werden. Zum Beispiel können alle Funktionen mit zwei Protagonisten gesucht werden.

Parameter:
+ Der erste Parameter ist die Korpussammlumg. Hier einfach *corpus_collection* stehen lassen.
+ Der zweite Parameter *count* ist die Anzahl, die gesucht werden soll
+ Der dritte Parameter ist die Kategorie, die *count* Mal vergeben wurde
+ Der letzte Parameter *export* (*True* oder *False*) gibt an, ob die Daten als Excel-Datei im derzeitigen Verzeichnis abgelegt werden sollen.

In [7]:
count_labels = filtering_corpus.count_category_instances(
    corpus_collection,
    2,
    "all_morals",
    export=False
)
util.print_subcorp_dict(count_labels)

GERICHTSURTEILE-POS-AW-NEU-OPTIMIERT-BB_VERGOLDET.XMI
Der Obstkonzern aus Cincinnati hatte die Zahlungen an die Milizen zugegeben, aber als notwendig verteidigt. Man habe DAS LEBEN bedrohter Mitarbeiter SCHÜTZEN müssen.
Das Gesetz, das zu einem solchen Eingriff ermächtigt, muss Vorkehrungen enthalten, um DEN KERNBEREICH PRIVATER LEBENSGESTALTUNG zu SCHÜTZEN.
Jüdische Gemeinden, die außerhalb des Zentralrats stehen, haben ebenfalls das Recht, PARITÄTISCH berücksichtigt zu werden. Anderenfalls werde DIE RELIGIÖS-WELTANSCHAULICHE NEUTRALITÄT verletzt.
Schon 2011 warnte Pell als Ehrengast der britischen klimaskeptischen Organisation Global Warming Policy Foundation (GWPF) vor der Moral, den Haushalten und Familien Kosten AUFZUBÜRDEN, ganz besonders den ARMEN, im Namen der Rettung des Planeten und in der Hoffnung, den Anstieg globaler Temperaturen zu bremsen.
Der EuGH ist hingegen der Ansicht, dass sich die Öffentlichkeit über MÖGLICHE GEFAHREN FÜR DIE UMWELT und DIE MENSCHLICHE GESUNDHEIT 

Mit der folgenden Funktion kann nach spezifischen Labeln gesucht werden werden. Zum Beispiel können alle Funktionen herausgesucht werden, in denen ein Protagonist als "Benefizient" markiert wurde.

Parameter:
+ Der erste Parameter ist die Korpussammlumg. Hier einfach *corpus_collection* stehen lassen.
+ Der zweite Parameter ist das Label, nach dem gesucht werden soll
+ Der dritte Parameter ist die Kategorie, aus der das Label stammt
+ Der letzte Parameter *export* (*True* oder *False*) gibt an, ob die Daten als Excel-Datei im derzeitigen Verzeichnis abgelegt werden sollen.

In [8]:
tag_labels = filtering_corpus.label_category_instances(
    corpus_collection,
    "Benefizient:in",
    "protagonists",
    export=False
)
util.print_subcorp_dict(tag_labels)

GERICHTSURTEILE-POS-AW-NEU-OPTIMIERT-BB_VERGOLDET.XMI
Im Anschluss an die Veröffentlichung der Studie hat sich die Kritik am Vorgehen der räuberischen Aktionäre in den vergangenen Wochen verstärkt. Denn der Grundsatz der Gleichbehandlung ALLER AKTIONÄRE verbietet, dass sich einzelne Sondervorteile versprechen lassen.
Zum Schutz DER BEVÖLKERUNG" sei dies dringend notwendig, so Beckstein.
Die Staatsanwaltschaft warf ihr vor, Vertrauen missbraucht zu haben. „PATIENTEN müssen sich darauf verlassen können, dass Ärzte über ein geprüftes Fachwissen verfügen.”
Die 2. Kammer des Ersten Senats sah sich gleichwohl – weil es noch um die Erstattung der Prozesskosten ging – nicht an einer inhaltlichen Entscheidung gehindert und brachte Grundrechtsdogmatik und gesunden Menschenverstand wieder in Einklang: Die Befreiung von der Gebühr sei ein Gebot der Gleichbehandlung, weil DIE BETROFFENEN trotz ihrer auf dem Papier höheren Bezüge am Ende schlechter dastünden als ein normaler Hartz-IV-Empfänger.
Das 

In [12]:
type_instances = filtering_corpus.type_category_instances(
    corpus_collection,
    "das",
    "all_morals",
    export=False,
    lower=True
)
util.print_subcorp_dict(type_instances)

GERICHTSURTEILE-POS-AW-NEU-OPTIMIERT-BB_VERGOLDET.XMI
Der griechische Zentralrat betont, das fundamentale Recht der Meinungsfreiheit dürfe nicht dazu dienen, Hass und Antisemitismus zu verbreiten.
Diese Wettbewerbsveränderung wollte der Bundesrat bereits 1988 abschaffen, scheiterte aber an der Bundesregierung, die ihrerseits nun vom Verfassungsgericht korrigiert wurde. Die Privilegierung der Parteien verletze das Recht auf Chancengleichheit und sei mit dem Grundgesetz unvereinbar. Die Steuerbefreiung müsse bis zu einer Neuregelung durch den Gesetzgeber auch auf kommunale Wählervereinigungen ausgedehnt werden.
Es) bleibt eine Schutzlücke, die durch das allgemeine Persönlichkeitsrecht in seiner Ausprägung als Schutz der Vertraulichkeit und Integrität von informationstechnischen Systemen zu schließen ist.
Dieser dürfe einzelnen Zuschauern nicht willkürlich den Zutritt zum Stadion verbieten, weil sowohl das allgemeine Persönlichkeitsrecht als auch das Gebot der Gleichbehandlung zu berücksi